<img src="https://raw.github.com/emsig/logos/main/empymod/empymod-logo.png" width="400" alt="Logo empymod" title="Logo empymod">
    

### 3D EM modeller for 1D VTI media

The modeller **empymod** can compute electric or magnetic responses due to a 3D
electric or magnetic source in a layered-earth model with vertical transverse
isotropic (VTI) resistivity, VTI electric permittivity, and VTI magnetic
permeability, from very low frequencies (DC) to very high frequencies (GPR).
The computation is carried out in the wavenumber-frequency domain, and various
Hankel- and Fourier-transform methods are included to transform the responses
into the space-frequency and space-time domains.


- Manual: https://empymod.emsig.xyz
- Gallery: https://empymod.emsig.xyz/en/stable/gallery
- Code: https://github.com/emsig/empymod
- Installation: https://empymod.emsig.xyz/en/stable/manual/installation.html

In [ ]:
# Uncomment on Google Colab
# %pip install empymod matplotlib ipympl ipywidgets scooby

In [ ]:
import empymod

import numpy as np
import matplotlib.pyplot as plt

import ipywidgets as widgets
from ipywidgets import interact

plt.style.use('ggplot')

# Comment this out on Google Colab
%matplotlib widget

## Routines

At its heart, empymod has the Green's functions for x-, y-, and z-directed infinitesimal small electric and magnetic dipole sources and receivers (36 combinations), using recursively local and global reflection coefficients. It is based on [Hunziker et al., 2015, Geophysics](https://doi.org/10.1190/geo2013-0411.1).
- It is analytical in the wavenumber-frequency domain.
- For the wavenumber-to-space and frequency-to-time transformations there are various transformation methods, where the digital linear filter (DLF) is the default approach.
- Analytical full- and half-space solution in the frequency- and time-domain.

### Primary routines in empymod:

- `bipole`
- `dipole`
- `loop`
- `analytical`
- `dipole_k`

In [ ]:
# Look at docs
empymod.bipole?

In [ ]:
empymod.bipole(
    src=(0, 0, -250, 0, 0),
    rec=(1000, 0, -300, 0, 0),
    depth=[0, -300, -1000, -1100],
    res=[2e14, 0.3, 1, 100, 1],
    freqtime=1,
    verb=3,
)

## Basic example with numpy and matplotlib
A simple frequency-domain example, where we keep most of the parameters left at the default value:

First we define the survey parameters: source and receiver locations, and source frequencies.

In [ ]:
# x-directed bipole source: x0, x1, y0, y1, z0, z1
source = [-50, 50, 0, 0, -250, -250]

# Source frequency
frequency_1 = 1

# Receiver offsets
offsets_1 = np.linspace(500, 10000, 301)

# x-directed dipole receiver-array: x, y, z, azimuth, dip
receivers_1 = [offsets_1, offsets_1*0, -300, 0, 0]

Next, we define the resistivity model:

In [ ]:
# Layer boundaries
depth = [0, -300, -1000, -1050]

# Layer resistivities
resistivities = [2e14, 0.3, 1, 50, 1]

And finally we compute the electromagnetic response at receiver locations:

In [ ]:
resp_1 = empymod.bipole(
    src=source,
    rec=receivers_1,
    depth=depth,
    res=resistivities,
    freqtime=frequency_1,
    htarg={'pts_per_dec': -1},
    verb=2,
)

Let’s plot the resulting responses:

In [ ]:
fig1, ((ax11, ax12), (ax13, ax14)) = plt.subplots(2, 2, sharex=True, constrained_layout=True)

fig1.suptitle(f'Electric response; freq={frequency_1} Hz')

ax11.set_title('|Re(E)| (V/m)')
ax11.semilogy(offsets_1/1e3, abs(resp_1.real))

ax12.set_title('|E| (V/m)')
ax12.semilogy(offsets_1/1e3, resp_1.amp())

ax13.set_title('|Im(E)| (V/m)')
ax13.semilogy(offsets_1/1e3, abs(resp_1.imag))

ax14.set_title('Pha(E) (°)')
ax14.plot(offsets_1/1e3, resp_1.pha(deg=True))

for ax in [ax13, ax14]:
    ax.set_xlabel('Offset (km)')

## As a function of frequency instead of offset

In [ ]:
frequency_2 = np.logspace(-2, 2, 200)
offsets_2 = 3000.0
receivers_2 = [offsets_2, offsets_2*0, -200, 0, 0]

resp_2 = empymod.bipole(
    src=source,
    rec=receivers_2,
    depth=depth,
    res=resistivities,
    freqtime=frequency_2,
    htarg={'pts_per_dec': -1},
    verb=2,
)

In [ ]:
fig2, ((ax21, ax22), (ax23, ax24)) = plt.subplots(2, 2, sharex=True, constrained_layout=True)

fig2.suptitle(f'Electric response; off={offsets_2/1e3} km')

ax21.set_title('|Re(E)| (V/m)')
ax21.loglog(frequency_2, abs(resp_2.real))

ax22.set_title('|E| (V/m)')
ax22.loglog(frequency_2, resp_2.amp())

ax23.set_title('|Im(E)| (V/m)')
ax23.loglog(frequency_2, abs(resp_2.imag))

ax24.set_title('Pha(E) (°)')
ax24.semilogx(frequency_2, resp_2.pha(deg=True))

for ax in [ax23, ax24]:
    ax.set_xlabel('Frequency (Hz)')

## Using input dictionaries

=> Makes it easy to compare different models keeping most parameters constant

We use this simple example also to play around with, e.g.:

- `signal`
- `msrc`, `mrec`
- `aniso`
- `eperm{H;V}`
- `mperm{H;V}`

In [ ]:
inp = {
    'src': source,
    #'src': (*source[:3], 90, 0),
    'rec': receivers_1,
    #'rec': (*receivers_1[:3], 90, 0),
    #'aniso': [1, 1, 2, 2, 2],
    'depth': depth,
    'freqtime': frequency_1,
    #'signal': 0,
    #'msrc': True,
    #'mrec': True,
    'htarg': {'pts_per_dec': -1},
    'verb': 1,
}

resp_1_tg = empymod.bipole(res=resistivities, **inp)
resp_1_bg = empymod.bipole(res=[2e14, 0.3, 1, 1, 1], **inp)
#resp_1_bg = empymod.bipole(res=resistivities, epermH=[1, 1, 80, 80, 80], **inp)

In [ ]:
fig3, ((ax31, ax32), (ax33, ax34)) = plt.subplots(2, 2, sharex=True, constrained_layout=True)

fig3.suptitle(f'Electric response; freq={frequency_1} Hz')

ax31.set_title('|Re(E)| (V/m)')
ax31.semilogy(offsets_1/1e3, abs(resp_1_tg.real), label='target')
ax31.semilogy(offsets_1/1e3, abs(resp_1_bg.real), label='background')
ax31.legend()

ax32.set_title('|E| (V/m)')
ax32.semilogy(offsets_1/1e3, resp_1_tg.amp())
ax32.semilogy(offsets_1/1e3, resp_1_bg.amp())

ax33.set_title('|Im(E)| (V/m)')
ax33.semilogy(offsets_1/1e3, abs(resp_1_tg.imag))
ax33.semilogy(offsets_1/1e3, abs(resp_1_bg.imag))

ax34.set_title('Pha(E) (°)')
ax34.plot(offsets_1/1e3, resp_1_tg.pha(deg=True))
ax34.plot(offsets_1/1e3, resp_1_bg.pha(deg=True))

for ax in [ax33, ax34]:
    ax.set_xlabel('Offset (km)')

### Save/load input & CLI

See https://empymod.emsig.xyz/en/stable/manual/iocli.html

In [ ]:
empymod.io.save_input('myrun.json', {'res': resistivities, **inp})

In [ ]:
inp_loaded = empymod.io.load_input('myrun.json')

In [ ]:
%%bash

empymod --help

In [ ]:
!cat myrun.json

In [ ]:
%%bash

empymod bipole myrun.json output.json

In [ ]:
empymod.io.load_data('output.json')

## Interactive

Small example how to create a simple interactive example.

In [ ]:
fig4, ax41 = plt.subplots()
ax41.set_ylabel('Amplitude (V/m)')
ax41.set_xlabel('Offset (m)')
ax41.set_yscale('log')
    
ax41.plot(offsets_1, resp_1_bg.amp(), label='Target')
ax41.plot(offsets_1, resp_1_bg.amp(), label='Background')
ax41.legend()
    
def f(res):
    update_resp = empymod.bipole(res=[2e14, 0.3, 1, 10**res, 1], **inp)
    ax41.set_title(f"Resistivity: {10**res: 6.2f} Ω m")
    ax41.lines[0].set_ydata(update_resp.amp())
    
interact(f, res=widgets.FloatSlider(min=-2, max=2, step=0.1, value=0));

## Other topics:

- TLE notebook (if time, run it!)
- WalkTEM & [Gallery](https://empymod.emsig.xyz/en/stable/gallery) in general
- Induced Polarization
- Loops
- Ziolkowski and Slob

## Keep in mind

empymod is very useful to quickly check ideas, concepts, and learn about the field.

However, keep in mind that it is only for layered models: You will almost always get an exagerated response in comparison with reality!

In [ ]:
# Check the report - can be useful
empymod.Report()